In [1]:
# Clean Apple product price lists across countries
# refer: https://www.kaggle.com/code/prasertk/clean-apple-product-price-lists-across-countries/notebook

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas_datareader import data
import re
import plotly.express as px

import ipywidgets as widgets
from ipywidgets import interact

 


In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/mohan-chinnappan-n/datasets/master/apple/apple%20product%20price%20list%20from%2026%20countries.csv')
df['country_code']=df['country_code'].str.upper()
df

,pid,model,price,country_code,country,region,income group,scraped_date
0,0,iPhone 13,"From A$1,199",AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535
1,1,iPhone SE,From A$679,AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535
2,2,iPhone 12,From A$999,AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535
3,3,AirPods(3rd generation),A$279,AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535
4,4,AirPods Pro,A$399,AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535
...,...,...,...,...,...,...,...,...
443,12,iPad Pro,From $799 or $66.58/mo. per month for 12 mo.*,US,United States,North America,High income,2022-01-02 21:52:47.021744
444,13,Apple Pencil (2nd generation),$129 or $10.75/mo. per month for 12 mo.*,US,United States,North America,High income,2022-01-02 21:52:47.021744
445,14,24-inch iMac²,From $1299 or $108.25/mo. per month for 12 mo.*,US,United States,North America,High income,2022-01-02 21:52:47.021744
446,15,MacBook Air,From $999 or $83.25/mo. per month for 12 mo.*,US,United States,North America,High income,2022-01-02 21:52:47.021744


## translate all foreign model names into English name (model_en column) 

In [4]:
df['model_en']=df['model']

df.loc[df['model'].str.contains('imac', case=False), 'model_en'] = '24-inch iMac'
df.loc[df['model'].str.contains('pen', case=False), 'model_en'] = 'Apple Pencil (2nd generation)'
df.loc[df['model'].str.contains('spor|Спортивный ремешок|Correa deportiva|Urheiluranneke', case=False), 'model_en'] = 'Sport Band'
df.loc[df['model'].str.contains('airpods\(2|airpods\(andra|airpods\(รุ่นที่ 2|airpods\(seconda|airpods\(segunda', case=False), 'model_en'] = 'AirPods(2nd generation)'
df.loc[df['model'].str.contains('airpods\(3|airpods\(รุ่นที่ 3|airpods\(tre|airpods\(ter', case=False), 'model_en'] = 'AirPods(3rd generation)'
df.loc[df['model'].str.contains('mouse', case=False), 'model_en'] = 'Magic Mouse'
df.loc[df['model'].str.contains('Apple Store|iTunes', case=False), 'model_en'] = 'App Store and iTunes Gift Card'

In [5]:
df

,pid,model,price,country_code,country,region,income group,scraped_date,model_en
0,0,iPhone 13,"From A$1,199",AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,iPhone 13
1,1,iPhone SE,From A$679,AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,iPhone SE
2,2,iPhone 12,From A$999,AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,iPhone 12
3,3,AirPods(3rd generation),A$279,AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,AirPods(3rd generation)
4,4,AirPods Pro,A$399,AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,AirPods Pro
...,...,...,...,...,...,...,...,...,...
443,12,iPad Pro,From $799 or $66.58/mo. per month for 12 mo.*,US,United States,North America,High income,2022-01-02 21:52:47.021744,iPad Pro
444,13,Apple Pencil (2nd generation),$129 or $10.75/mo. per month for 12 mo.*,US,United States,North America,High income,2022-01-02 21:52:47.021744,Apple Pencil (2nd generation)
445,14,24-inch iMac²,From $1299 or $108.25/mo. per month for 12 mo.*,US,United States,North America,High income,2022-01-02 21:52:47.021744,24-inch iMac
446,15,MacBook Air,From $999 or $83.25/mo. per month for 12 mo.*,US,United States,North America,High income,2022-01-02 21:52:47.021744,MacBook Air


In [6]:
df['model_en'].value_counts()


Sport Band                        26
MacBook Air                       26
iPhone 12                         26
AirPods Pro                       26
AirPods(2nd generation)           26
Magic Mouse                       26
Apple TV 4K                       26
Apple Watch SE                    26
Apple Watch Series 3              26
iPhone SE                         26
iPad                              26
iPad Pro                          26
Apple Pencil (2nd generation)     26
24-inch iMac                      26
AirPods(3rd generation)           25
iPhone 13                         21
Apple TV HD                       17
AirPods Max                       11
iPhone 13 Pro                      5
App Store and iTunes Gift Card     5
Name: model_en, dtype: int64

## Extrat Starting Price

In [7]:
df['price2']=df['price'].str.split('or', 1, expand=True)[0]
df

,pid,model,price,country_code,country,region,income group,scraped_date,model_en,price2
0,0,iPhone 13,"From A$1,199",AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,iPhone 13,"From A$1,199"
1,1,iPhone SE,From A$679,AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,iPhone SE,From A$679
2,2,iPhone 12,From A$999,AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,iPhone 12,From A$999
3,3,AirPods(3rd generation),A$279,AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,AirPods(3rd generation),A$279
4,4,AirPods Pro,A$399,AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,AirPods Pro,A$399
...,...,...,...,...,...,...,...,...,...,...
443,12,iPad Pro,From $799 or $66.58/mo. per month for 12 mo.*,US,United States,North America,High income,2022-01-02 21:52:47.021744,iPad Pro,From $799
444,13,Apple Pencil (2nd generation),$129 or $10.75/mo. per month for 12 mo.*,US,United States,North America,High income,2022-01-02 21:52:47.021744,Apple Pencil (2nd generation),$129
445,14,24-inch iMac²,From $1299 or $108.25/mo. per month for 12 mo.*,US,United States,North America,High income,2022-01-02 21:52:47.021744,24-inch iMac,From $1299
446,15,MacBook Air,From $999 or $83.25/mo. per month for 12 mo.*,US,United States,North America,High income,2022-01-02 21:52:47.021744,MacBook Air,From $999


In [8]:
# keep only digits, . (period), and , (comma)
df['price2']=df['price2'].str.replace('[^\d\.\,]', '', regex=True)
df

,pid,model,price,country_code,country,region,income group,scraped_date,model_en,price2
0,0,iPhone 13,"From A$1,199",AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,iPhone 13,"1,199"
1,1,iPhone SE,From A$679,AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,iPhone SE,679
2,2,iPhone 12,From A$999,AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,iPhone 12,999
3,3,AirPods(3rd generation),A$279,AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,AirPods(3rd generation),279
4,4,AirPods Pro,A$399,AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,AirPods Pro,399
...,...,...,...,...,...,...,...,...,...,...
443,12,iPad Pro,From $799 or $66.58/mo. per month for 12 mo.*,US,United States,North America,High income,2022-01-02 21:52:47.021744,iPad Pro,799
444,13,Apple Pencil (2nd generation),$129 or $10.75/mo. per month for 12 mo.*,US,United States,North America,High income,2022-01-02 21:52:47.021744,Apple Pencil (2nd generation),129
445,14,24-inch iMac²,From $1299 or $108.25/mo. per month for 12 mo.*,US,United States,North America,High income,2022-01-02 21:52:47.021744,24-inch iMac,1299
446,15,MacBook Air,From $999 or $83.25/mo. per month for 12 mo.*,US,United States,North America,High income,2022-01-02 21:52:47.021744,MacBook Air,999


In [9]:
# take care of different decimal separators

comma_decimal_point_locale = ['AM', 'AR', 'AT', 'AZ', 'BA', 'BE', 'BG', 'BO', 'BR', 'BY',
                    'CA', 'CH', 'CL', 'CM', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK',
                    'EC', 'EE', 'ES', 'FI', 'FO', 'FR', 'GE', 'GL',  'HR', 'HU',
                    'ID', 'IS', 'IT', 'KZ', 'LB', 'LT', 'LU', 'LV', 'MA', 'MD', 'MK', 'MO', 'MZ',
                    'NL', 'NO', 'PE', 'PL', 'PT', 'PY', 'RO', 'RS', 'RU',
                    'SE', 'SI', 'SK', 'TN', 'TR', 'UA', 'UY', 'UZ', 'VE', 'VN', 'ZA']

price_clean=[]
for i, r in df.iterrows():
    p=r['price2']
    if r['country_code'] in comma_decimal_point_locale:
        p=p.replace('.', '').replace(',', '.')
    else:
        p=p.replace(',', '')
    price_clean.append(p)
    
df['price_local_currency']=price_clean
df['price_local_currency']=pd.to_numeric(df['price_local_currency'])
df


,pid,model,price,country_code,country,region,income group,scraped_date,model_en,price2,price_local_currency
0,0,iPhone 13,"From A$1,199",AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,iPhone 13,"1,199",1199.0
1,1,iPhone SE,From A$679,AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,iPhone SE,679,679.0
2,2,iPhone 12,From A$999,AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,iPhone 12,999,999.0
3,3,AirPods(3rd generation),A$279,AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,AirPods(3rd generation),279,279.0
4,4,AirPods Pro,A$399,AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,AirPods Pro,399,399.0
...,...,...,...,...,...,...,...,...,...,...,...
443,12,iPad Pro,From $799 or $66.58/mo. per month for 12 mo.*,US,United States,North America,High income,2022-01-02 21:52:47.021744,iPad Pro,799,799.0
444,13,Apple Pencil (2nd generation),$129 or $10.75/mo. per month for 12 mo.*,US,United States,North America,High income,2022-01-02 21:52:47.021744,Apple Pencil (2nd generation),129,129.0
445,14,24-inch iMac²,From $1299 or $108.25/mo. per month for 12 mo.*,US,United States,North America,High income,2022-01-02 21:52:47.021744,24-inch iMac,1299,1299.0
446,15,MacBook Air,From $999 or $83.25/mo. per month for 12 mo.*,US,United States,North America,High income,2022-01-02 21:52:47.021744,MacBook Air,999,999.0


In [10]:
df['country'].unique()

array(['Australia', 'Austria', 'Brazil', 'Canada', 'Czech Republic',
       'Germany', 'Denmark', 'Spain', 'Finland', 'France', 'Hungary',
       'India', 'Ireland', 'Italy', 'Luxembourg', 'Mexico', 'Netherlands',
       'Norway', 'Philippines', 'Poland', 'Portugal', 'Russia', 'Sweden',
       'Thailand', 'Turkey', 'United States'], dtype=object)

In [11]:
@interact
def filter(regex_pat="."):
    return df[(df["country"].str.contains(regex_pat, regex=True, case=False))].sort_values(
        "pid", ascending=True
    )

interactive(children=(Text(value='.', description='regex_pat'), Output()), _dom_classes=('widget-interact',))

In [12]:
# get exchange rate
dc=pd.read_csv('https://raw.githubusercontent.com/mohan-chinnappan-n/datasets/master/apple/exchange-rates/countryAndCurrencyCode.csv')
dc=dc.drop_duplicates(subset=['Alpha2 Code']).reset_index(drop=True)
dc

,Currency Name,Country Name,Alpha2 Code,Currency CodeA,Currency CodeN,Minor Unit
0,Afghani,AFGHANISTAN,AF,AFN,971,2
1,Algerian Dinar,ALGERIA,DZ,DZD,12,2
2,Ariary,MADAGASCAR,MG,MGA,969,0
3,Argentine Peso,ARGENTINA,AR,ARS,32,2
4,Armenian Dram,ARMENIA,AM,AMD,51,2
...,...,...,...,...,...,...
231,Yemeni Rial,YEMEN,YE,YER,886,2
232,Yen,JAPAN,JP,JPY,392,0
233,Yuan Renminbi,CHINA,CN,CNY,156,2
234,Zimbabwe Dollar,ZIMBABWE,ZW,ZWD,716,2


In [13]:
df=pd.merge(df, dc[['Alpha2 Code', 'Currency CodeA']], how='inner', left_on='country_code', right_on='Alpha2 Code').drop(columns=['Alpha2 Code'])
df

,pid,model,price,country_code,country,region,income group,scraped_date,model_en,price2,price_local_currency,Currency CodeA
0,0,iPhone 13,"From A$1,199",AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,iPhone 13,"1,199",1199.0,AUD
1,1,iPhone SE,From A$679,AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,iPhone SE,679,679.0,AUD
2,2,iPhone 12,From A$999,AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,iPhone 12,999,999.0,AUD
3,3,AirPods(3rd generation),A$279,AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,AirPods(3rd generation),279,279.0,AUD
4,4,AirPods Pro,A$399,AU,Australia,East Asia & Pacific,High income,2022-01-02 21:52:06.630535,AirPods Pro,399,399.0,AUD
...,...,...,...,...,...,...,...,...,...,...,...,...
443,12,iPad Pro,From $799 or $66.58/mo. per month for 12 mo.*,US,United States,North America,High income,2022-01-02 21:52:47.021744,iPad Pro,799,799.0,USD
444,13,Apple Pencil (2nd generation),$129 or $10.75/mo. per month for 12 mo.*,US,United States,North America,High income,2022-01-02 21:52:47.021744,Apple Pencil (2nd generation),129,129.0,USD
445,14,24-inch iMac²,From $1299 or $108.25/mo. per month for 12 mo.*,US,United States,North America,High income,2022-01-02 21:52:47.021744,24-inch iMac,1299,1299.0,USD
446,15,MacBook Air,From $999 or $83.25/mo. per month for 12 mo.*,US,United States,North America,High income,2022-01-02 21:52:47.021744,MacBook Air,999,999.0,USD


In [ ]:
# ref: https://pandas-datareader.readthedocs.io/en/latest/remote_data.html#remote-data-yahoo
dfx=data.DataReader(df['Currency CodeA'].unique() + '=X', 'yahoo', 
                    start=(pd.Timestamp.now() + pd.Timedelta(days=-5)).strftime('%Y-%m-%d'), end=pd.Timestamp.now().strftime('%Y-%m-%d'))['Close'].stack()
dfx=dfx.to_frame().reset_index()
dfx.columns=['FX_Date', 'Currency CodeA', 'X']
dfx['Currency CodeA']=dfx['Currency CodeA'].str[:3]
dfx=dfx.drop_duplicates(subset=['Currency CodeA'], keep='last')
dfx